In [1]:
# Update over all code

import sys
import pygame

from time import sleep
from game_stats import GameStats
from settings import Settings
from ship import Ship 
from bullet import Bullet
from alien import Alien
from button import Button
from difficulty import Difficulty
from scoreboard import Scoreboard

class AlienInvasion:
    """Model the overall class to manage the game assets and behaviors"""
    
    def __init__(self):
        """Initialize the game, create the game resources"""
        
        pygame.init()
        
        self.settings = Settings()
        self.screen = pygame.display.set_mode((0,0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion by Khanhdz2k2")
        
        # Create an instance to store the game statistics
        self.stats = GameStats(self)
        
        # Adding the ship attribute by calling Ship() instance
        self.ship = Ship(self)
        
        # Adding the bullets group by pygame.sprite.Group() function
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        
        # Adding the play button instance
        self.play_button = Button(self, "Play")
        
        # Adding the difficulty buttons instance
        self.difficulty_button = Difficulty(self, "Easy", "Medium", "Hard", "Extreme")
        
        # Adding the scoreboard instance
        self.sb = Scoreboard(self)
        
        self._create_fleet()
    
    def run_game(self):
        """Start the main loop for the game"""
        
        while True:
            self._check_events()
            
            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
            
            self._update_screen()
    def _fire_bullet(self):
        """Fire the bullet"""
        
        # Fire a bullet if the number of existing bullets is lesser than the bullets_allowed
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
    
    def _create_fleet(self):
        """Create a fleet of aliens"""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        # Determine the number of aliens can fit horizontally
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width) 
        
        # Determine the number of rows can fit on the screen
        ship_height = self.ship.rect.height
        available_space_y = self.settings.screen_height - (2.5 * alien_height) - ship_height
        number_rows = available_space_y // (1.25 * alien_height)
        
        number_of_rows = int(number_rows)
        
        # Create the first row of aliens.
        for row_number in range(number_of_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)
    
    def _create_alien(self, alien_number, row_number):
        """Create an alien and place it in the row."""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        alien.x = alien_width + (2 * alien_width * alien_number)
        alien.rect.x = alien.x
        
        alien.y = (0.25 * alien_height) + (1.25*alien_height*row_number)
        alien.rect.y = alien.y
        
        self.aliens.add(alien)        
    
    def _check_fleet_edges(self):
        """Response appropriately if the alien hits the edges"""
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
    
    def _change_fleet_direction(self):
        """Drop the entire fleet and change its direction"""
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
    
    def _ship_hit(self):
        """Respond to the ship thats been hit by an alien"""
        if self.stats.ship_left > 0:
            # Decrement ship left
            self.stats.ship_left -= 1

            # Delete all remaining bullets and aliens
            self.bullets.empty()
            self.aliens.empty()

            # Respawn the ship and the fleet
            self._create_fleet()
            self.ship.center_ship()

            # Pauses the game for 1 second
            sleep(1)
            
        else:
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
    
    def _check_events(self):
        """Responses keyboard and mouse events"""
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)
                self._check_difficulty_button(mouse_pos)
            
    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        
        # Move the ship to the right when pressing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        # Move the ship to the left when pressing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        # Exit the game with q press
        elif event.key == pygame.K_q:
            pygame.quit()
            sys.exit()
        # Fire bulelts with spacebar
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
        elif event.key == pygame.K_p and self.stats.game_active == False:
            self._start_game()
        
        
    def _check_keyup_events(self, event):
        """Response to key releases"""
        
        # Stop the ship when releasing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        # Stop the ship when releasing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
    
    def _check_play_button(self, mouse_pos):
        """Start a new game when the player click play"""
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and self.stats.game_active == False:
            self._start_game()
            self.settings.initialize_dynamic_settings()
    
    def _check_difficulty_button(self, mouse_pos):
        """Choose the level of difficulty"""
        
        easy_button_clicked = self.difficulty_button.easy_rect.collidepoint(mouse_pos)
        medium_button_clicked = self.difficulty_button.medium_rect.collidepoint(mouse_pos)
        hard_button_clicked = self.difficulty_button.hard_rect.collidepoint(mouse_pos)
        extreme_button_clicked = self.difficulty_button.extreme_rect.collidepoint(mouse_pos)
        
        if easy_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.easy_button_color = (105,105,105)
            self.settings.difficulty = 'easy'
       
        elif medium_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.medium_button_color = (105,105,105)
            self.settings.difficulty = 'medium'
            
        elif hard_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.hard_button_color = (105,105,105)
            self.settings.difficulty = 'hard'
            
        elif extreme_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.extreme_button_color = (105,105,105)
            self.settings.difficulty = 'extreme'
        
        self.difficulty_button._prep_msg('Easy', 'Medium', 'Hard', 'Extreme')
    
    def _start_game(self):
        """Start the game"""
        # Reset the game stats
        self.stats.reset_stats()
        self.stats.game_active = True

        # Get rid of any remaining aliens or bullets
        self.aliens.empty()
        self.bullets.empty()

        # Respawn the fleet and center the ship
        self._create_fleet()
        self.ship.center_ship()
            
        # Hide mouse cursor ingame
        pygame.mouse.set_visible(False)
    
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
        then update the positions of all aliens in the fleet.
        """
        # Check if a alien reaches the edges
        self._check_fleet_edges()
        self.aliens.update()
        
        # Detect any ship and aliens collisions
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            self._ship_hit()
        
        # Check if any alien reaches the bottom of the screen
        self._check_aliens_bottom()
        
    def _update_bullets(self):
        """Update position of the bullets and get rid of old bullets"""
        
        # Update bullet position
        self.bullets.update()
        
        # Get rid of old bullets
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)
         
        # Check for bullet and alien collisions
        self._check_bullet_alien_collisions()
        
    def _check_bullet_alien_collisions(self):
        """Check for bullet and alien collisions"""
        
        # Delete any bullet and alien that has collided
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, True, True)
        
        if not self.aliens:
            # Delete the remaining bullets and create a new fleet
            self.bullets.empty()
            self._create_fleet()
            self.settings.increase_speed()
    
    def _check_aliens_bottom(self):
        """Check if any alien reaches the bottom of the screen"""
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                self._ship_hit()
                break
    
    def _update_screen(self):
        """Update images on the screen, and flip to the next screen"""
         
        # Redrawn the screen during each pass through the loop
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)    
        
        # Draw the scoreboard to the screen
        self.sb.show_score()
        
        # Draw the play button if the game is inactive
        if not self.stats.game_active:
            self.play_button.draw_button()
        
        # Draw the difficulty button if the game is inactive
        
        if not self.stats.game_active:
            self.difficulty_button.draw_button()
        
        # Make the most recently drawn screen visible
        pygame.display.flip()
        
if __name__ == '__main__':
    # Make a game instance and run the game
    ai = AlienInvasion()
    ai.run_game()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Acer Phong Vu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Scoring
Implement a scoring system to track real time score, highscore, level and number of ships remaining

In [ ]:
# game_stats.py

def reset_stats(self):
    --snip--
    self.score = 0

- To reset the score each time a new game start, we put self.score attribute in reset_stats() method not in __init__ () method which will only reset the score once when the game is turned on

# Displaying the score

In [ ]:
# scoreboard.py

import pygame.font

class Scoreboard:
    """A class to report scoring information"""
    
    def __init__(self, ai_game):
        """Initialize the scoreboard attributes"""
        self.screen = ai_game.screen
        self.screen_rect = self.screen.get_rect()
        self.settings = ai_game.settings
        self.stats = ai_game.stats
        
        # Font settings for scoring information
        self.text_color = (30, 30, 30)
        self.font = pygame.font.SysFont(None, 48)
        
        # Prepare the initial scoring image
        self.prep_score()
        
    def prep_score(self):
        """Turn the score into rendered image"""
        self.score_str = str(self.stats.score)
        self.score_image = self.font.render(score_str, True, self.text_color, self.settings.bg_color)
        
        # Display the score on the top right of the screen
        self.score_rect = self.score_image.get_rect()
        self.score_rect.right = self.screen.right - 20
        self.score_rect.top = 20
        
    def show_score(self):
        """Draw the score the the screen"""
        self.screen,blit(self.score_image, self.score_rect)

To display the score on the screen, we create a new class Scoreboard
- Because Scoreboard writes text on the screen, we initially import pygame.font
- We give __init__() ai_game parameter to get access to screen, settings, stats attributes 
- Then we set a text color and create a font object
- To call the text to be displayed as an image, we call prep_score()
    - We assign the score attribute in stats as a string then pass it to render() function which creates the image of that string
    - To make the score clear on screen, we use the screen bg_color as the text background color
    - The position the score on the top right of the screen and make it expands to the left when the value increases
    - We make it 20 pixel off the screen.right and 20 pixel off top of the screen
- Finally, we define a method to draw the score to the screen at the location score_rect is located called show_score()

In [ ]:
# Making a scoreboard

# alien_invasion.py

from scoreboard import Scoreboard

--snip--

# Create a scoreboard
self.sb = Scoreboard(self)

- We first import the Scoreboard to alien_invasion and make an instance of it in __init__()


In [ ]:
# alien_invasion.py

def _update_screen(self):
    --snip--
    # Draw the score board 
    self.sb.show_score()

- Then we draw the score board on the screen in _update_screen() method


In [ ]:
# Updating the score as aliens are shot down

# settings.py
def initialize_dynamic_settings(self):
    --snip--
    self.alien_points = 50

# alien_invasion.py

def _check_bullet_alien_collisions(self):
    --snip--
    if collisions:
        self.stats.score += self.settings.alien_points
        self.sb.prep_score()

To write live score on screen, we update the value of stats.score whenever an alien is shot down and call prep_score() to update the score image
- We will increase the point value as the game progresses. To make sure this point value is reset each time the game start over, we put it in initialize_dynamic_settings() method
- In _check_bullet_alien_collisions() method, we detect if there are any dictionary returned by Pygame from the collision.
    - We check if the dictionary exist and if it does, we add points to our score attribute
    - Then we call prep_score() to create new image for updated score

In [ ]:
# Reseting the score

def _check_play_button(self):
    --snip--
    if button_clicked and self.stats.game_active == False:
        --snip--
        self.sb.prep_score()

- We still see the old score when a new game is created
- To solve this, we call prep_score() whenever the play button is clicked which prep the score board with 0 score

In [ ]:
# Make sure to score all hits

# alien_invasion.py
def _check_bullet_alien_collisions(self):
    --snip--
    for aliens in collisions.values():
        self.stats.score += self.settings.alien_points * len(aliens)

- Currently our code is missing some point when the bullets hit the aliens
- For example, 2 bullets hit aliens during the same pass through the loop or an extra large bullet hits all aliens at once still count as 1 alien hit
- To fix this, we refine the detect of how alien are scored
- In collisions dictionary, any aliens hit is considered a value in the dictionary. So we will loop through these values to make sure we point all the aliens hit
    - If collision dictionary is defined, we loop through every value in it
    - Each value is associated with 1 key of the dictionary which is the bullet, so we can multiply the points of an alien with the number of alien in the dictionary



In [ ]:
# Increasing point values

# settings.py

    self.easy_score_scale = 1.4
    self.medium_score_scale = 1.6
    self.hard_score_scale = 1.8
    self.extreme_score_scale = 2

def increase_speed(self):
    --snip--
    self.alien_points = int(self.alien_points * self.score_scale)

- Each level the game will become more difficult so adding more points to an alien hit when at different level differently is appropriate
- We first define a rate which points increase so whenever the game speed increases, the points is also increase
- int() funtion is to define the point value as an interger

In [ ]:
# Rounding the score

# scoreboard_py

def prep_score(self):
    rounded_score = round(self.stats.score, -1)
    score_str = "{:,}".format(rounded_score)

As we develop our game in arcade style, we might as well want the score to be multiple of 10, and there should be a comma seperates the number
- The round() function usually round the number of a decimal value with the provided place in the second argument
    - However, when passing a negative argument, this function will round the nearest 10, 100, 1000 and so on 
    - The above code tells python to round seld.stats.score to the nearest 10 and store it in rounded_score
- A string formatting directive tells python to insert comma into number when converting a numerical value into string
    - For example, for the output 1000000 will be 1,000,000

In [ ]:
# High scores

# game_stats.py

def __init__(self, ai_game):
    --snip--
    # High score should never be reset
    self.high_score = 0

Because highscore should never be reset, we store it in __init__ method

In [ ]:
# scoreboard.py

def __init__(self, ai_game):
    --snip--
    # Prepare for the initial score image
    self.prep_score()
    self.prep_high_score()
    
def prep_high_score(self):
    """Prepare for the highscore into rendered image"""
    rounded_high_score = round(self.stats.high_score, -1)
    high_score_str = "{:,}".format(rounded_high_score)
    self.high_score_image = self.font.render(high_score_str, True, self.text_color, self.settings.bg_color)
    
    # Center the highscore at the top right of the screen
    self.high_score_rect = self.high_score_image.get_rect()
    self.high_score_rect.centerx = self.scren_rect.centerx
    self.high_score_rect.top = self.score_rect.top 

- We round the highscore to the nearest 10 and add commas to it
- We then generate image from the highscore, center the highscore rect attribute horizontally and set its top attribute match with the score top attribute


In [ ]:
# scoreboard.py

def show_score(self):
    --snip--
    self.screen.blit(self.high_score_image, self.high_score_rect)
    
def check_high_score(self):
    """Detect the high score"""
    if self.stats.score > self.stats.high_score:
        self.stats.high_score = self.stats.score
        self.prep_high_score()

- The show_score() method now show the highscore as well
- To detect any new highscore, we write an additional method called check_high_score()
    - if the current score > current highscore, it shall be the new highscore
    - We call prep_high_score() method to update this highscore

In [ ]:
# alien_invasion.py

def _check_bullet_alien_collisions:
    --snip--
    if collisions:
        for aliens in collisions.values():
            self.stats.score += self.settings.alien_points * len(aliens)
            self.sb.prep_score()
            self.sb.check_high_score()

We call for check_high_score after detect the collisions dictionary and after update all the points from aliens that have been shot down

In [1]:
# Update over all code

import sys
import pygame

from time import sleep
from game_stats import GameStats
from settings import Settings
from ship import Ship 
from bullet import Bullet
from alien import Alien
from button import Button
from difficulty import Difficulty
from scoreboard import Scoreboard

class AlienInvasion:
    """Model the overall class to manage the game assets and behaviors"""
    
    def __init__(self):
        """Initialize the game, create the game resources"""
        
        pygame.init()
        
        self.settings = Settings()
        self.screen = pygame.display.set_mode((0,0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion by Khanhdz2k2")
        
        # Create an instance to store the game statistics
        self.stats = GameStats(self)
        
        # Adding the ship attribute by calling Ship() instance
        self.ship = Ship(self)
        
        # Adding the bullets group by pygame.sprite.Group() function
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        
        # Adding the play button instance
        self.play_button = Button(self, "Play")
        
        # Adding the difficulty buttons instance
        self.difficulty_button = Difficulty(self, "Easy", "Medium", "Hard", "Extreme")
        
        # Adding the scoreboard instance
        self.sb = Scoreboard(self)
        
        self._create_fleet()
    
    def run_game(self):
        """Start the main loop for the game"""
        
        while True:
            self._check_events()
            
            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
            
            self._update_screen()
    def _fire_bullet(self):
        """Fire the bullet"""
        
        # Fire a bullet if the number of existing bullets is lesser than the bullets_allowed
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
    
    def _create_fleet(self):
        """Create a fleet of aliens"""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        # Determine the number of aliens can fit horizontally
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width) 
        
        # Determine the number of rows can fit on the screen
        ship_height = self.ship.rect.height
        available_space_y = self.settings.screen_height - (2.5 * alien_height) - ship_height
        number_rows = available_space_y // (1.25 * alien_height)
        
        number_of_rows = int(number_rows)
        
        # Create the first row of aliens.
        for row_number in range(number_of_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)
    
    def _create_alien(self, alien_number, row_number):
        """Create an alien and place it in the row."""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        alien.x = alien_width + (2 * alien_width * alien_number)
        alien.rect.x = alien.x
        
        alien.y = alien_height + (1.25 * alien_height * row_number)
        alien.rect.y = alien.y
        
        self.aliens.add(alien)        
    
    def _check_fleet_edges(self):
        """Response appropriately if the alien hits the edges"""
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
    
    def _change_fleet_direction(self):
        """Drop the entire fleet and change its direction"""
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
    
    def _ship_hit(self):
        """Respond to the ship thats been hit by an alien"""
        if self.stats.ship_left > 0:
            # Decrement ship left
            self.stats.ship_left -= 1

            # Delete all remaining bullets and aliens
            self.bullets.empty()
            self.aliens.empty()

            # Respawn the ship and the fleet
            self._create_fleet()
            self.ship.center_ship()

            # Pauses the game for 1 second
            sleep(1)
            
        else:
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
    
    def _check_events(self):
        """Responses keyboard and mouse events"""
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)
                self._check_difficulty_button(mouse_pos)
            
    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        
        # Move the ship to the right when pressing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        # Move the ship to the left when pressing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        # Exit the game with q press
        elif event.key == pygame.K_q:
            pygame.quit()
            sys.exit()
        # Fire bulelts with spacebar
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
        elif event.key == pygame.K_p and self.stats.game_active == False:
            self._start_game()
        
        
    def _check_keyup_events(self, event):
        """Response to key releases"""
        
        # Stop the ship when releasing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        # Stop the ship when releasing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
    
    def _check_play_button(self, mouse_pos):
        """Start a new game when the player click play"""
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and self.stats.game_active == False:
            self._start_game()
            self.settings.initialize_dynamic_settings()
            self.sb.prep_score()
            self.sb.prep_level()
    
    def _check_difficulty_button(self, mouse_pos):
        """Choose the level of difficulty"""
        
        easy_button_clicked = self.difficulty_button.easy_rect.collidepoint(mouse_pos)
        medium_button_clicked = self.difficulty_button.medium_rect.collidepoint(mouse_pos)
        hard_button_clicked = self.difficulty_button.hard_rect.collidepoint(mouse_pos)
        extreme_button_clicked = self.difficulty_button.extreme_rect.collidepoint(mouse_pos)
        
        if easy_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.easy_button_color = (105,105,105)
            self.settings.difficulty = 'easy'
       
        elif medium_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.medium_button_color = (105,105,105)
            self.settings.difficulty = 'medium'
            
        elif hard_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.hard_button_color = (105,105,105)
            self.settings.difficulty = 'hard'
            
        elif extreme_button_clicked and self.stats.game_active == False:
            self.difficulty_button.re_color_button()
            self.difficulty_button.extreme_button_color = (105,105,105)
            self.settings.difficulty = 'extreme'
        
        self.difficulty_button._prep_msg('Easy', 'Medium', 'Hard', 'Extreme')
    
    def _start_game(self):
        """Start the game"""
        # Reset the game stats
        self.stats.reset_stats()
        self.stats.game_active = True

        # Get rid of any remaining aliens or bullets
        self.aliens.empty()
        self.bullets.empty()

        # Respawn the fleet and center the ship
        self._create_fleet()
        self.ship.center_ship()
            
        # Hide mouse cursor ingame
        pygame.mouse.set_visible(False)
    
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
        then update the positions of all aliens in the fleet.
        """
        # Check if a alien reaches the edges
        self._check_fleet_edges()
        self.aliens.update()
        
        # Detect any ship and aliens collisions
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            self._ship_hit()
        
        # Check if any alien reaches the bottom of the screen
        self._check_aliens_bottom()
        
    def _update_bullets(self):
        """Update position of the bullets and get rid of old bullets"""
        
        # Update bullet position
        self.bullets.update()
        
        # Get rid of old bullets
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)
         
        # Check for bullet and alien collisions
        self._check_bullet_alien_collisions()
        
    def _check_bullet_alien_collisions(self):
        """Check for bullet and alien collisions"""
        
        # Delete any bullet and alien that has collided
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, True, True)
        
        if not self.aliens:
            # Delete the remaining bullets and create a new fleet
            self.bullets.empty()
            self._create_fleet()
            self.settings.increase_speed()
            
            # Level increment
            self.stats.level += 1
            self.sb.prep_level()
        
        if collisions:
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
                self.sb.prep_score()
                self.sb.check_high_score()
    
    def _check_aliens_bottom(self):
        """Check if any alien reaches the bottom of the screen"""
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                self._ship_hit()
                break
    
    def _update_screen(self):
        """Update images on the screen, and flip to the next screen"""
         
        # Redrawn the screen during each pass through the loop
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)    
        
        # Draw the scoreboard to the screen
        self.sb.show_score()
        
        # Draw the play button if the game is inactive
        if not self.stats.game_active:
            self.play_button.draw_button()
        
        # Draw the difficulty button if the game is inactive
        
        if not self.stats.game_active:
            self.difficulty_button.draw_button()
        
        # Make the most recently drawn screen visible
        pygame.display.flip()
        
if __name__ == '__main__':
    # Make a game instance and run the game
    ai = AlienInvasion()
    ai.run_game()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Acer Phong Vu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Displaying the level

In [ ]:
# game_stats.py

def reset_stats(self):
    --snip--
    self.level = 1

- We first define an attribute to represent level 1 in game_stats
- We reset the level every time we restart the game, so we put this in reset_stats()

In [ ]:
# scoreboard.py

def __init__(self, ai_game):
    --snip--
    self.prep_level()

def prep_level(self):
    """Turn the level into rendered image"""
    level = self.stats.level
    level_str = str(level)
    self.level_image = self.font.render(level_str, True, self.text_color, self.settings.bg_color)
    
    # Display the level rect below the score
    self.level_rect = self.level_image.get_rect()
    self.level_rect.right = self.score_rect.right
    self.level_rect.top = self.score_rect.bottom + 10

def show_score(self):
    --snip--
    self.screen.blit(self.level_image, self.level_rect)

- The prep_level() method create the level image and display it on the screen below the score image
- Then we draw it to the screen

In [ ]:
# alien_invasion.py

def _check_bullet_alien_collisions(self):
    if not self.aliens:
        --snip--
        # Level increment
        self.stats.level += 1
        self.sb.prep_level()
    
def _check_play_button(self, mouse_pos):
    --snip--  
    if button_clicked and not self.stats.game_active:
        --snip--
        self.sb.prep_score()
        self.sb.prep_level()

- Everytime the whole fleet is destroyed, we increment the level by 1 and call prep_level() to make sure all levels are updated
- To ensure the level are reset to 1 every time a new game is started, we call prep_level() after prep_score() when the Play button is clicked

# Displaying the number of ships

- We use graphic to display the amount of ships we have 
- To do so, we draw the ships graphic on the top left of the screen represent our lives

In [ ]:
# lives.py

import pygame
from pygame.sprite import Sprite

class Live(Sprite):
    def __init__(self, ai_game):
        super().__init__()
        self.screen = ai_game.screen
        self.screen_rect = self.screen.get_rect()
        
        # Load the image of heart 
        self.image = pygame.image.load('images/lives.bmp').convert()
        self.rect = self.image.get_rect()

- We import Sprite and make sure Ship class inherits it to create a group of ships
- In __init__ method, we call super()

In [ ]:
# scoreboard.py

from pygame.sprite import Group
from life import Life

def __init__(self, ai_game):
    self.ai_game = ai_game
    --snip--
    self.prep_lives()
    
def prep_lives(self):
    """Show how many ships are left"""
    self.lives = Group()
    for life_number in range(self.stats.ships_left):
        life = Life(self.ai_game)
        ship.rect.x = 10 + life_number * life.rect.width
        ship.rect.y = 10
        self.lives.add(life)

def show_score(self):
    --snip--
    self.lives.draw(self.screen)

- We assign the game instance to an attribute, because we will need to pass it later on and we call a new method prep_ship()
- In prep_ship(), we create an empty group call self.ships
    - Then we create a loop to loop through all the ships the player has left
    - Inside the loop we create an instance of Ship and pass self.ai_game as an argument
    - The ship is located at the top left of the screen with a 10 pixels margin
    - We set each ship right next to each other
    - Then we add the new ship to the group ships
- Then we draw the ships group using draw() in show_score method

In [ ]:
# alien_invasion

def _check_play_button(self, mouse_pos):
    --snip--
    self.sb.prep_lives()
    
def _ship_hit(self):
    """Respond to ship being hit by alien."""
        if self.stats.ships_left > 0:
        # Decrement ships_left, and update scoreboard.
        self.stats.ships_left -= 1
        self.sb.prep_lives()
        --snip--

- We call prep_ships() in _check_play_button() method to show the player how many ships they start with 
- We also call prep_ship() after decrese ships_left when the ship is hit to minus 1 ship from the player and update the display of ships left

In [1]:
# Final main code

import sys
import pygame
import json

from time import sleep
from game_stats import GameStats
from settings import Settings
from ship import Ship 
from bullet import Bullet
from alien import Alien
from button import Button
from difficulty import Difficulty
from scoreboard import Scoreboard
from life import Life
from power_shot import PowerShot
from explosion import Explosion
from sound import SoundFX


class AlienInvasion:
    """Model the overall class to manage the game assets and behaviors"""
    
    def __init__(self):
        """Initialize the game, create the game resources"""
        
        pygame.init()
        
        self.settings = Settings()
        self.screen = pygame.display.set_mode((0,0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion by Khanhdz2k2")
        
        # Create an instance to store the game statistics
        self.stats = GameStats(self)
        
        # Adding the ship attribute by calling Ship() instance
        self.ship = Ship(self)
        
        # Adding the groups by pygame.sprite.Group() function
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        self.power_shots = pygame.sprite.Group()
        self.bullet_explosions = pygame.sprite.Group()
        self.power_shot_explosions = pygame.sprite.Group()
        
        # Adding the play button instance
        self.play_button = Button(self, "Play", 668, 407)
        
        # Adding the difficulty buttons instance
        self.difficulty_button = Difficulty(self, "Easy", "Medium", "Hard", "Extreme")
        
        # Adding the menu button instance
        self.resume_button = Button(self, "Resume", 668, 300)
        self.quit_button = Button(self, "Quit", 668, 500)
        
        # Adding the scoreboard instance
        self.sb = Scoreboard(self)    
        
        # Create the first fleet of aliens
        self._create_fleet()
        
        # Create the sound effects instance
        self.soundfx = SoundFX(self)
    
    def run_game(self):
        """Start the main loop for the game"""
        
        while True:
            self._check_events()
            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
            self._update_screen()
    
    def _fire_bullet(self):
        """Fire the bullet"""
        
        # Fire a bullet if the number of existing bullets is lesser than the bullets_allowed
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
    
    def _fire_power_shot(self):
        """Fire the power shot"""
        
        # Fire a power shot due to the number of remaining power shots
        if self.settings.power_shots_number:
            new_power_shot = PowerShot(self)
            self.power_shots.add(new_power_shot)
            self.soundfx.power_shot_fx.play()
            self.settings.power_shots_number -= 1
    
    def _create_fleet(self):
        """Create a fleet of aliens"""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        # Determine the number of aliens can fit horizontally
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width) 
        
        # Determine the number of rows can fit on the screen
        ship_height = self.ship.rect.height
        available_space_y = self.settings.screen_height - (2.5 * alien_height) - ship_height
        number_rows = available_space_y // (1.25 * alien_height)
        
        number_of_rows = int(number_rows)
        
        # Create the first row of aliens.
        for row_number in range(number_of_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)
    
    def _create_alien(self, alien_number, row_number):
        """Create an alien and place it in the row."""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        alien.x = alien_width + (2 * alien_width * alien_number)
        alien.rect.x = alien.x
        
        alien.y = alien_height + (1.25 * alien_height * row_number)
        alien.rect.y = alien.y
        
        self.aliens.add(alien)        
    
    def _check_fleet_edges(self):
        """Response appropriately if the alien hits the edges"""
        
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
    
    def _change_fleet_direction(self):
        """Drop the entire fleet and change its direction"""
        
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
    
    def _check_aliens_bottom(self):
        """Check if any alien reaches the bottom of the screen"""
        
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                self._ship_hit()
                self.soundfx.ship_explosion_fx.play()
                break
    
    def _ship_hit(self):
        """Respond to the ship thats been hit by an alien"""
        
        self.soundfx.ship_explosion_fx.play()
        if self.stats.ships_left > 0:
            # Decrement ship left
            self.stats.ships_left -= 1
            self.sb.prep_lives()

            # Delete all remaining bullets and aliens
            self.bullets.empty()
            self.aliens.empty()

            # Respawn the ship and the fleet
            self._create_fleet()
            self.ship.center_ship()

            # Pauses the game for 1 second
            sleep(1)
            
        else:
            # Display game over!
            self.stats.game_over = True
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
            self.soundfx.game_over_fx.play()
            
    
    def _check_events(self):
        """Responses keyboard and mouse events"""
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)
                self._check_difficulty_button(mouse_pos)
                self._check_pause(mouse_pos)
    
    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        
        # Move the ship to the right when pressing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        # Move the ship to the left when pressing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        # Exit the game with q press
        elif event.key == pygame.K_q:
            self._store_high_score()
            pygame.quit()
            sys.exit()
        # Fire bulelts with spacebar
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
        # Fire power shots with b press
        elif event.key == pygame.K_b:
            self._fire_power_shot()
        # Pause the game with p press
        elif event.key == pygame.K_p :
            self.stats.game_pause = True
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
        
        
    def _check_keyup_events(self, event):
        """Response to key releases"""
        
        # Stop the ship when releasing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        # Stop the ship when releasing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
    
    def _check_pause(self, mouse_pos):
        """Check if the game is pause"""
        
        resume_button_clicked = self.resume_button.rect.collidepoint(mouse_pos)
        quit_button_clicked = self.quit_button.rect.collidepoint(mouse_pos)
        if resume_button_clicked:
            self.stats.game_active = True
            self.stats.game_pause = False
        elif quit_button_clicked:
            pygame.quit()
            sys.exit()
    
    def _check_play_button(self, mouse_pos):
        """Start a new game when the player click play"""
        
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and self.stats.game_active == False:
            self._start_game()
            self.settings.initialize_dynamic_settings()
            self.sb.prep_images()
    
    def _check_difficulty_button(self, mouse_pos):
        """Choose the level of difficulty"""
        
        easy_button_clicked = self.difficulty_button.easy_rect.collidepoint(mouse_pos)
        medium_button_clicked = self.difficulty_button.medium_rect.collidepoint(mouse_pos)
        hard_button_clicked = self.difficulty_button.hard_rect.collidepoint(mouse_pos)
        extreme_button_clicked = self.difficulty_button.extreme_rect.collidepoint(mouse_pos)
        
        # Re-color the buttons after click another
        self.difficulty_button.re_color_button()
        
        if easy_button_clicked and self.stats.game_active == False:
            self.difficulty_button.easy_button_color = (105,105,105)
            self.settings.difficulty = 'easy'
       
        elif medium_button_clicked and self.stats.game_active == False:
            self.difficulty_button.medium_button_color = (105,105,105)
            self.settings.difficulty = 'medium'
        
        elif hard_button_clicked and self.stats.game_active == False:
            self.difficulty_button.hard_button_color = (105,105,105)
            self.settings.difficulty = 'hard'
            
        elif extreme_button_clicked and self.stats.game_active == False:
            self.difficulty_button.extreme_button_color = (105,105,105)
            self.settings.difficulty = 'extreme'
        
        # Update the button color after each click
        self.difficulty_button._prep_msg('Easy', 'Medium', 'Hard', 'Extreme')
    
    def _start_game(self):
        """Start the game"""
        
        # Reset the game stats
        self.stats.reset_stats()
        self.stats.game_active = True
        self.stats.game_over = False

        # Get rid of any remaining aliens or bullets
        self.aliens.empty()
        self.bullets.empty()

        # Respawn the fleet and center the ship
        self._create_fleet()
        self.ship.center_ship()
            
        # Hide mouse cursor ingame
        pygame.mouse.set_visible(False)
    
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
        then update the positions of all aliens in the fleet.
        """
        # Check if a alien reaches the edges
        self._check_fleet_edges()
        self.aliens.update()
        
        # Detect any ship and aliens collisions
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            self._ship_hit()
        
        # Check if any alien reaches the bottom of the screen
        self._check_aliens_bottom()
        
    def _update_bullets(self):
        """Update position of the bullets and get rid of old bullets"""
        
        # Update bullet position
        self.bullets.update()
        # Update bullet explosion effect
        self.bullet_explosions.update()
        
        # Update power_shot position
        self.power_shots.update()
        # Update power shot explosion effect
        self.power_shot_explosions.update()
        
        # Get rid of old bullets and power shots
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)
         
        for power_shot in self.power_shots.copy():
            if power_shot.rect.bottom <= 0:
                self.power_shots.remove(power_shot)
                
        
        # Check for bullet and alien collisions
        self._check_bullet_alien_collisions()
        self._check_power_shot_alien_collisions()
        
    
    def _check_bullet_alien_collisions(self):
        """Check for bullet and alien collisions"""
        
        # Delete any bullet and alien that has collided
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, True, True)
        
        # Start new level when all aliens are shot down
        if not self.aliens:
                self._start_new_level()
            
        if collisions:
            # Play the sound effect
            self.soundfx.alien_explosion_fx.play()
            
            # Add the bullet explosion instance
            for bullet in collisions.keys():
                explosion = Explosion(self, 'bullet', bullet.rect.centerx, bullet.rect.centery)
                self.bullet_explosions.add(explosion)
            
            # Counts and scores all aliens hit 
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
                self.sb.prep_score()
                self.sb.check_high_score()
                
                   
            
    def _check_power_shot_alien_collisions(self):
        """Check for power shot and alien collisions"""
        # Delete any alien hit by the powershot
        collisions = pygame.sprite.groupcollide(self.power_shots, self.aliens, False, True)
        
        if collisions:
            # Play the sound effect
            self.soundfx.alien_explosion_fx.play()
            
            # Add the power shot explosion instance
            for power_shot in collisions.keys():
                explosion = Explosion(self, 'power_shot', power_shot.rect.x, power_shot.rect.y)
                self.power_shot_explosions.add(explosion)
            
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
                self.sb.prep_score()
                self.sb.check_high_score()
        
    def _start_new_level(self):
        """Start the new level when whole fleet is destroyed"""
        
        # Delete the remaining bullets and create a new fleet
        self.bullets.empty()
        self._create_fleet()
        self.settings.increase_speed()
            
        # Level increment
        self.stats.level += 1
        self.sb.prep_level()
        
    def _store_high_score(self):
        """Save the highest score recorded"""
        
        filename = 'highscore.json'
        with open(filename, 'w') as file_object:
            json.dump(self.sb.high_score, file_object)
    
    def _update_screen(self):
        """Update images on the screen, and flip to the next screen"""
         
        # Redrawn the screen during each pass through the loop
        self.screen.blit(self.settings.bg, (0,0))
        self.ship.blitme()
        
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)    
        
        for power_shot in self.power_shots.sprites():
            power_shot.draw_power_shot()
        
        for explosion in self.bullet_explosions.sprites():
            explosion.draw_explosion()
            
        for explosion in self.power_shot_explosions.sprites():
            explosion.draw_explosion()
        # Draw the scoreboard to the screen
        self.sb.show_score()
        
        # Draw the play button if the game is inactive  
        # Draw the difficulty button if the game is inactive
        if not self.stats.game_active:
            if not self.stats.game_pause:
                self.difficulty_button.draw_button()
                self.play_button.draw_button()
        
        # Draw the menu if the game is pause
            elif self.stats.game_pause == True:
                self.resume_button.draw_button()
                self.quit_button.draw_button()
        
        if self.stats.game_over == True:
            self.screen.blit(self.sb.game_over_image, self.sb.game_over_rect)
        
        # Make the most recently drawn screen visible
        pygame.display.flip()
        
if __name__ == '__main__':
    # Make a game instance and run the game
    ai = AlienInvasion()
    ai.run_game()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Acer Phong Vu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
